#Import libary

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from keras.models import Model
from keras import models, layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import copy

from google.colab import files
from google.colab import drive
from functools import partial
drive.mount('/content/drive')

In [ ]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
function ClickConnect(){
  btn = document.querySelector("colab-connect-button")
  if (btn != null){
    console.log("Click colab-connect-button"); 
    btn.click() 
    }
    
  btn = document.getElementById('ok')
  if (btn != null){
    console.log("Click reconnect"); 
    btn.click() 
    }
  }
    
setInterval(ClickConnect,60000)
'''))

In [ ]:
#import file input
nameFile = 'A_grid_Group_fixed_missing_values_and_normalization.csv'
data = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/Data_normalization/'+nameFile)
data_grid_name = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/Data_normalization/'+nameFile)


In [ ]:
#cut the name of grid
data_grid_name = data.columns.tolist()[1:]
#cut the name of day
data_date_name = data['start_date']
#data input drop column start date
data = data.drop(columns=['start_date']) 
#tranform data to np.arr
data = np.asarray(data,dtype=np.float32) 

#Data tranform


In [ ]:
#input data 24hr after (taget day)
#label 24hr before (taget day)
#every grid in group

def data_transform(x_data, before, next, grid):

  X, y = list(), list()
  data_name = list()
  count = 9
  for i in range(0,x_data.shape[0],24):
    for r in range(grid):
      temp = None
      seq_X = None
      seq_y = None
      past = None
      
      if i > before-1 and x_data.shape[0] - i > next:
        
        temp = copy.deepcopy(x_data[i][:8])

        past = [x_data[i-x][count-1] for x in range(1,before+1)]

        temp = temp.tolist()
        temp.extend(past)

        seq_X = np.asarray(temp)

        temp = [x_data[i+x][count-1] for x in range(next)]

        temp_ = []
        temp_.append(data_date_name[i+1])
        if int(x_data[i][1]) == 1:
          temp_.append('Monday')
        elif int(x_data[i][2]) == 1:
          temp_.append('Tuesday')
        elif int(x_data[i][3]) == 1:
          temp_.append('Wednesday')
        elif int(x_data[i][4]) == 1:
          temp_.append('Thursday')
        elif int(x_data[i][5]) == 1:
          temp_.append('Friday')
        elif int(x_data[i][6]) == 1:
          temp_.append('Saturday')
        elif int(x_data[i][7]) == 1:
          temp_.append('Sunday')
        temp_.append(data_grid_name[count-1])
        data_name.append(temp_)

        seq_y = np.asarray(temp)
        X.append(seq_X)
        y.append(seq_y)
        count += 1
        if count > len(x_data[0]):
          count = 9

  x_array = np.asarray(X)
  y_array = np.asarray(y)
  data_name = np.asarray(data_name)
  return x_array, y_array, data_name

In [ ]:
#tranforn v2

def convert_to_model(x_data, grid):
  label_index = 8
  x_array = list()
  y_array = list()
  data_name = list()
  for round in range(label_index,label_index+grid):
    for index in range(24,len(x_data)-24):
      y_array.append(x_data[index][round])
      temp = list()
      for x in range(1,4):
        temp.append(x_data[index-x][round])
      for xx in range(21,25):
        temp.append(x_data[index-xx][round])
      temp = np.array(temp)
      x_array.append(np.append(x_data[index][:8].copy(),temp,0))


      temp_ = []
      temp_.append(data_date_name[index])
      if int(x_data[index][1]) == 1:
        temp_.append('Monday')
      elif int(x_data[index][2]) == 1:
        temp_.append('Tuesday')
      elif int(x_data[index][3]) == 1:
        temp_.append('Wednesday')
      elif int(x_data[index][4]) == 1:
        temp_.append('Thursday')
      elif int(x_data[index][5]) == 1:
        temp_.append('Friday')
      elif int(x_data[index][6]) == 1:
        temp_.append('Saturday')
      elif int(x_data[index][7]) == 1:
        temp_.append('Sunday')
      temp_.append(data_grid_name[round])
      data_name.append(temp_)

  x_array = np.asarray(x_array)
  y_array = np.asarray(y_array)
  data_name = np.asarray(data_name)
  return x_array, y_array, data_name

In [ ]:
#24hr before pre
before = 24
#predice 24hr next 
next = 24 
#num of grid in group (del date and time)
grid = len(data[0])-8 

data_input,data_label,data_name =  convert_to_model(data, grid=grid)



#Neural network Model Buide 

In [ ]:
#buide model
model = Sequential()
model.add(Dense(100, activation="elu", input_shape=(data_input.shape[1:])))

model.add(Dropout(0.1))
model.add(Dense(100, activation="elu"))
model.add(Dropout(0.1))
model.add(Dense(100, activation="elu"))
model.add(Dropout(0.1))
model.add(Dense(100, activation="elu"))
model.add(Dropout(0.1))
model.add(Dense(100, activation="elu"))
model.add(Dropout(0.1))
model.add(Dense(1, activation='linear'))

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
adam =tf.keras.optimizers.Adam(learning_rate=0.001,name='Adam')
model.compile(optimizer=adam, loss='mae')

model.summary()

#Training Data

In [ ]:
#day start to train data
day = 50
start = day*grid*24

#day start to test data
day_left = (len(data_input)-start) 
true_plot = list()
predict_plot = list()
name_plot = list()
r = 0

for round in range(0,int(day_left),24):
  train_input = data_input[:start]
  train_label = data_label[:start]
  model.fit(train_input, train_label, epochs=1,callbacks=[callback])
  predict_plot_aux = list()
  true_plot_aux = list()
  for step in range(24):
    predict_plot_aux.append(model.predict(data_input[start+step].reshape(1,-1))[0][0])
    true_plot_aux.append(data_label[start+step])

  name_plot.append(data_name[start+step])
  predict_plot.append(predict_plot_aux)
  true_plot.append(true_plot_aux)
  start += 24

In [ ]:
model.predict(data_input[100].reshape(1,-1))[0][0]

#Scale up output 

In [ ]:

mean_ = 3
std_ = 4

true_plot_scaleup = copy.deepcopy(true_plot)
predict_plot_scaleup = copy.deepcopy(predict_plot)

for item in range(len(true_plot_scaleup)):
  for ii in range(24):
    true_plot_scaleup[item][ii] = ((true_plot_scaleup[item][ii]*std_)+mean_).round()
    predict_plot_scaleup[item][ii] = ((predict_plot_scaleup[item][ii]*std_)+mean_).round()
  

In [ ]:
#output data
len(predict_plot_scaleup)
len(true_plot_scaleup)

#Save prediction output to csv file

In [ ]:
name_data = pd.DataFrame(name_plot)
name_data.columns = ['start_date','start_day','grid_ID']

In [ ]:
pred = pd.DataFrame(predict_plot_scaleup)
true = pd.DataFrame(true_plot_scaleup)

In [ ]:
output_pred_A = pd.merge(name_data, pred, left_index=True, right_index=True)
output_true_A = pd.merge(name_data, true, left_index=True, right_index=True)

In [ ]:
output_pred_A.to_csv(r'output_pred_A_lv2.csv', index = False)
output_true_A.to_csv(r'output_true_A_lv2.csv', index = False)

#Evaluation (Accuracy measurement)

##plot grid pickup prediction 

In [ ]:
    list_over_est = [[],[],[],[],[],[],[]]
    list_under_est = [[],[],[],[],[],[],[]]
    list_zero_acc = [[],[],[],[],[],[],[]]
    list_hr = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    list_hr_zero_all = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    list_hr_zero_acc = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    plot_est = [[],[],[],[],[],[],[]]
    plot_hr = []
    a=[]
    b=[]
    
    for g in range(len(true_plot_scaleup)):
      sum_predict = 0
      sum_true = 0
      plt.figure(figsize=(10, 6))
      plt.plot(true_plot_scaleup[g], label='Actual')
      plt.plot(predict_plot_scaleup[g], label='Predicted')
      plt.title(f"{name_plot[g][0]} {name_plot[g][1]} {name_plot[g][2]}")
      plt.xlabel('Hr')
      plt.ylabel('Pick_up')
      plt.legend()
      plt.show()
      
      confusion_matrix = np.array([[0, 0, 0],[0, 0, 0],[0,0,0]])

      sum_true = sum(true_plot_scaleup[g])
      sum_predict = sum(predict_plot_scaleup[g])
      a.append(sum_true)
      b.append(sum_predict)

      for h in range(len(true_plot_scaleup[g])):
        if true_plot_scaleup[g][h] <= 0 and predict_plot_scaleup[g][h] <= 0: #TN ไม่มีแล้วบอกว่าไม่มี True Negative
          confusion_matrix[0][0] += 1
        elif true_plot_scaleup[g][h] <= 0 and predict_plot_scaleup[g][h] > 0: #FP ไม่มีแล้วบอกว่ามี False Positive
          confusion_matrix[0][1] += 1
        elif true_plot_scaleup[g][h] > 0 and predict_plot_scaleup[g][h] <= 0: #FN มีแล้วบอกว่าไม่มี False Negative
          confusion_matrix[1][0] += 1
        elif true_plot_scaleup[g][h] > 0 and predict_plot_scaleup[g][h] > 0: #TP มีแล้วบอกว่ามี True Positive
          confusion_matrix[1][1] += 1
      confusion_matrix[0][2] = confusion_matrix[0][0]+confusion_matrix[0][1]
      confusion_matrix[1][2] = confusion_matrix[1][0]+confusion_matrix[1][1]
      confusion_matrix[2][0] = confusion_matrix[0][0]+confusion_matrix[1][0]
      confusion_matrix[2][1] = confusion_matrix[0][1]+confusion_matrix[1][1]
      
      confusion_matrix_pd = pd.DataFrame(confusion_matrix, ['Actual: NO','Actual: YES','  '], ['Predicted: NO','Predicted: YES','  '])
      print(f"confusion matrix of {name_plot[g][0]} {name_plot[g][1]} {name_plot[g][2]} : \n")
      print(confusion_matrix_pd)
      
      true_plot_copy = copy.deepcopy(true_plot_scaleup)
      predict_plot_copy = copy.deepcopy(predict_plot_scaleup)
      true_filter = []
      predict_filter = []
      zero_predict = []
      
      for ze in range(len(true_plot_copy[g])):
        if true_plot_copy[g][ze] == 0:
          if predict_plot_copy[g][ze] == 0:
            list_hr_zero_acc[ze] += 1
          list_hr_zero_all[ze] += 1
          zero_predict.append(predict_plot_copy[g][ze])
        else:
          temp = true_plot_copy[g][ze] - predict_plot_copy[g][ze]
          if temp > 0:
            list_hr[ze].append([temp,ze,"over estimate"])
          elif temp < 0:
            list_hr[ze].append([temp,ze,"under estimate"])
          else:
            list_hr[ze].append([temp,ze,"over estimate"])
            list_hr[ze].append([temp,ze,"under estimate"])
          true_filter.append(true_plot_copy[g][ze])
          predict_filter.append(predict_plot_copy[g][ze])
      


      err = np.array(true_filter) - np.array(predict_filter)
      err_negative = list(filter(lambda x: x<=0, err))
      err_positive = list(filter(lambda x: x>=0, err))
      zero_acc = list(filter(lambda x: x==0,zero_predict))
    
      if len(err_negative) == 0:
        err_negative = 0
      else:
        err_negative = (sum(err_negative)/len(err_negative))
      if len(err_positive) == 0:
        err_positive = 0
      else:
        err_positive = (sum(err_positive)/len(err_positive))
      if len(zero_acc) == 0:
        zero_acc = 0
      else:
        zero_acc = (len(zero_acc)/len(zero_predict))
      print(f"\nzero accurracy of {name_plot[g][0]} {name_plot[g][1]} {name_plot[g][2]} : {zero_acc}")
      print(f"err_positive of {name_plot[g][0]} {name_plot[g][1]} {name_plot[g][2]} : {err_positive}")
      print(f"err_negative of {name_plot[g][0]} {name_plot[g][1]} {name_plot[g][2]} : {err_negative}\n\n")
      print(name_plot[g][1])
      if name_plot[g][1] == "Monday":
        plot_est[0].append([err_positive,name_plot[g][1],"over estimate"])
        plot_est[0].append([err_negative,name_plot[g][1],"under estimate"])

        list_over_est[0].append([err_positive,name_plot[g][1]])
        list_under_est[0].append([err_negative,name_plot[g][1]])
        list_zero_acc[0].append([zero_acc,name_plot[g][1]])
      elif name_plot[g][1] == "Tuesday":
        plot_est[1].append([err_positive,name_plot[g][1],"over estimate"])
        plot_est[1].append([err_negative,name_plot[g][1],"under estimate"])
        
        list_over_est[1].append([err_positive,name_plot[g][1]])
        list_under_est[1].append([err_negative,name_plot[g][1]])
        list_zero_acc[1].append([zero_acc,name_plot[g][1]])
      elif name_plot[g][1] == "Wednesday":
        plot_est[2].append([err_positive,name_plot[g][1],"over estimate"])
        plot_est[2].append([err_negative,name_plot[g][1],"under estimate"])

        list_over_est[2].append([err_positive,name_plot[g][1]])
        list_under_est[2].append([err_negative,name_plot[g][1]])
        list_zero_acc[2].append([zero_acc,name_plot[g][1]])
      elif name_plot[g][1] == "Thursday":
        plot_est[3].append([err_positive,name_plot[g][1],"over estimate"])
        plot_est[3].append([err_negative,name_plot[g][1],"under estimate"])

        list_over_est[3].append([err_positive,name_plot[g][1]])
        list_under_est[3].append([err_negative,name_plot[g][1]])
        list_zero_acc[3].append([zero_acc,name_plot[g][1]])
      elif name_plot[g][1] == "Friday":
        plot_est[4].append([err_positive,name_plot[g][1],"over estimate"])
        plot_est[4].append([err_negative,name_plot[g][1],"under estimate"])

        list_over_est[4].append([err_positive,name_plot[g][1]])
        list_under_est[4].append([err_negative,name_plot[g][1]])
        list_zero_acc[4].append([zero_acc,name_plot[g][1]])
      elif name_plot[g][1] == "Saturday":
        plot_est[5].append([err_positive,name_plot[g][1],"over estimate"])
        plot_est[5].append([err_negative,name_plot[g][1],"under estimate"])

        list_over_est[5].append([err_positive,name_plot[g][1]])
        list_under_est[5].append([err_negative,name_plot[g][1]])
        list_zero_acc[5].append([zero_acc,name_plot[g][1]])
      elif name_plot[g][1] == "Sunday":
        plot_est[6].append([err_positive,name_plot[g][1],"over estimate"])
        plot_est[6].append([err_negative,name_plot[g][1],"under estimate"])

        list_over_est[6].append([err_positive,name_plot[g][1]])
        list_under_est[6].append([err_negative,name_plot[g][1]])
        list_zero_acc[6].append([zero_acc,name_plot[g][1]])
      print(f"true all -> {sum_true}")
      print(f"predict all -> {sum_predict}")
    
    plt.figure(figsize=(10, 6))
    plt.plot(a, label='Actual')
    plt.plot(b, label='Predicted')
    plt.title(f"all of predicted")
    plt.xlabel('all')
    plt.ylabel('Pick_up')
    plt.legend()
    plt.show()

    for l in range(len(list_hr_zero_acc)):
      list_hr_zero_acc[l] = list_hr_zero_acc[l] / list_hr_zero_all[l]

    over_est = list_over_est[0]+list_over_est[1]+list_over_est[2]+list_over_est[3]+list_over_est[4]+list_over_est[5]+list_over_est[6]
    under_est = list_under_est[0]+list_under_est[1]+list_under_est[2]+list_under_est[3]+list_under_est[4]+list_under_est[5]+list_under_est[6]
    zero_acc = list_zero_acc[0]+list_zero_acc[1]+list_zero_acc[2]+list_zero_acc[3]+list_zero_acc[4]+list_zero_acc[5]+list_zero_acc[6]
    plot_est = plot_est[0]+plot_est[1]+plot_est[2]+plot_est[3]+plot_est[4]+plot_est[5]+plot_est[6]

    for k in list_hr:
      plot_hr += k

    print(over_est)
    print(under_est)
    print(zero_acc)
    

##All avg pickup prediction plot

In [ ]:
    fig, ax = plt.subplots(figsize=(10, 6))
    plt.plot(all_true, label='Actual')
    plt.plot(all_predice, label='Predicted')
    plt.title("Data prediction (A group)")
    plt.xlabel('24hr in 36day predice')
    plt.ylabel('Avg pick-up per day')
    plt.show()


##Accuracy prediction pick-up in week 

In [ ]:
plot_day = pd.DataFrame(plot_est,columns=['pickup estimate','Day of the week','Estimate'])
plot_day_zero = pd.DataFrame(zero_acc,columns=['Accuracy','Day of the week'])
plot_hr_1 = pd.DataFrame(plot_hr,columns=['pickup estimate','Hour','Estimate'])
plot_hr_2 = pd.DataFrame(list_hr_zero_acc,columns=['zeroacc'])


In [ ]:
plot_day.to_csv(r'dayEstimate_A_lv2.csv', index = False)
plot_day_zero.to_csv(r'dayZero_A_lv2.csv', index = False)
plot_hr_1.to_csv(r'hrEstimate_A_lv2.csv', index = False)
plot_hr_2.to_csv(r'hrZero_A_lv2.csv', index = False)

In [ ]:
! zip A_model2.zip *.csv
! rm *.csv

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))
sns.set_theme(style="whitegrid")
sns.set(rc={'figure.figsize':(12,9)})
ax = sns.boxplot(x="Day of the week", y="pickup estimate", hue="Estimate", data=plot_day,palette="Set3",linewidth=1,showfliers = False)
plt.title("Estimate per day (A group)model2")

In [ ]:
name = f'Estimate per day_Agroup.png'
fig.savefig(name)

##Zero accuracy prediction pick-up in week 


In [ ]:
fig, ax = plt.subplots()
sns.set_theme(style="whitegrid")
sns.set(rc={'figure.figsize':(12,9)})
ax = sns.boxplot(x="Day of the week", y="Accuracy", data=plot_day_zero,palette="Set3",linewidth=1,showfliers = False)
plt.title("zero accuracy per day (A group)model2")


In [ ]:
name = f'Zero accuracy per day_Agroup.png'
fig.savefig(name)

##Accuracy prediction pick-up in hour

In [ ]:
fig, ax = plt.subplots()
sns.set_theme(style="whitegrid")
sns.set(rc={'figure.figsize':(25,15)})
ax = sns.boxplot(x="Hour", y="pickup estimate", hue="Estimate", data=plot_hr_1,palette="Set3",linewidth=1,showfliers = False)
plt.title("estimate per hr (A group)model2")

In [ ]:
name = f'Estimate per hour_Agroup.png'
fig.savefig(name)

##Zero accuracy prediction pick-up in week 


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plt.plot(list_hr_zero_acc, label='Actual')
plt.title(f"zero accuracy per hour A group model2")
plt.xlabel('Hour')
plt.xticks(range(0, 24))
plt.ylabel('accuracy%')
plt.show()

In [ ]:
name = f'zero accuracy per hour.png'
fig.savefig(name)

In [ ]:
! zip grid_predice_a.zip *.png
! rm *.png

##CSVFILE

In [ ]:
avg_e_d_a = plot_day.groupby(["Day of the week","Estimate"])["Avg pickup estimate"].mean().reset_index(name="mean")
avg_e_d_a
avg_e_d_a.to_csv(r'avg_e_d_a.csv', index = False)

In [ ]:
avg_zc_d_a = plot_day_zero.groupby("Day of the week")["Accuracy"].mean().reset_index(name="mean")
avg_zc_d_a.to_csv(r'avg_zc_d_a.csv', index = False)

In [ ]:
avg_e_hr_a = plot_hr_1.groupby(["Hour","Estimate"])["Avg pickup estimate"].mean().reset_index(name="mean")
avg_e_hr_a.to_csv(r'avg_e_hr_a.csv', index = False)

In [ ]:
avg_zero_hr_a = pd.DataFrame(data=list_hr_zero_acc)
avg_zero_hr_a.to_csv(r'avg_zero_hr_a.csv', index = False)

In [ ]:
! zip avg_predice_a.zip *.csv
! rm *.csv